In [23]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import ForwardDiff as FD
import MeshCat as mc 
using Test
using Random
import Convex as cvx 
import ECOS      # the solver we use in this hw 
# import Hypatia # other solvers you can try
# import COSMO   # other solvers you can try 
using ProgressMeter

# using Pkg
# Pkg.add("ProgressMeter")
include(joinpath(@__DIR__,"utils/rendezvous.jl"))

  Activating environment at `~/Desktop/16-745/Assignments/HW2_S24-main/Project.toml`


thruster_model (generic function with 1 method)

In [24]:
# utilities for converting to and from vector of vectors <-> matrix 
function mat_from_vec(X::Vector{Vector{Float64}})::Matrix
    # convert a vector of vectors to a matrix 
    Xm = hcat(X...)
    return Xm 
end
function vec_from_mat(Xm::Matrix)::Vector{Vector{Float64}}
    # convert a matrix into a vector of vectors 
    X = [Xm[:,i] for i = 1:size(Xm,2)]
    return X 
end

vec_from_mat (generic function with 1 method)

In [25]:
function create_dynamics(dt::Real)::Tuple{Matrix,Matrix}
    mu = 3.986004418e14 # standard gravitational parameter
    a = 6971100.0       # semi-major axis of ISS
    n = sqrt(mu/a^3)    # mean motion

    # continuous time dynamics ẋ = Ax + Bu
    A = [0     0  0    1   0   0; 
         0     0  0    0   1   0;
         0     0  0    0   0   1;
         3*n^2 0  0    0   2*n 0;
         0     0  0   -2*n 0   0;
         0     0 -n^2  0   0   0]
         
    B = Matrix([zeros(3,3);0.1*I(3)])

    nx = size(A, 1)
    nu = size(B, 2)

    # TODO: convert to discrete time X_{k+1} = Ad*x_k + Bd*u_k

    matrix = [A B; zeros(nu, nx + nu)]
    exp_matrix = exp(matrix *dt)

    Ad = exp_matrix[1:nx, 1:nx]
    Bd = exp_matrix[1:nx, nx+1:nx+nu]


    return Ad, Bd
end

create_dynamics (generic function with 1 method)

In [28]:
@testset "discrete dynamics" begin 
    A,B = create_dynamics(1.0)
    
    x = [1,3,-.3,.2,.4,-.5]
    u = [-.1,.5,.3]
    
    # test these matrices 
    @test isapprox(A*x + B*u, [1.195453, 3.424786, -0.78499972, 0.190925, 0.4495759, -0.4699993], atol = 1e-3)
    @test isapprox(det(A), 1, atol = 1e-8)
    @test isapprox(norm(B,Inf), 0.0999999803, atol = 1e-5)
    
end

Test Summary:     | Pass  Total
discrete dynamics |    3      3


Test.DefaultTestSet("discrete dynamics", Any[], 3, false, false)

In [38]:

"""
`u = convex_mpc(A,B,X_ref_window,xic,xg,u_min,u_max,N_mpc)`

setup and solve the above optimization problem, returning the 
first control u_1 from the solution (should be a length nu 
Vector{Float64}).  
"""
function convex_mpc(A::Matrix, # discrete dynamics matrix A
                    B::Matrix, # discrete dynamics matrix B
                    X_ref_window::Vector{Vector{Float64}}, # reference trajectory for this window 
                    xic::Vector, # current state x 
                    xg::Vector, # goal state 
                    u_min::Vector, # lower bound on u 
                    u_max::Vector, # upper bound on u 
                    N_mpc::Int64,  # length of MPC window (horizon)
                    )::Vector{Float64} # return the first control command of the solved policy 
    
    # get our sizes for state and control
    nx,nu = size(B)
    
    #check sizes 
    @assert size(A) == (nx, nx)
    @assert length(xic) == nx 
    @assert length(xg) == nx 
    @assert length(X_ref_window) == N_mpc 
        
    # LQR cost
    Q = diagm(ones(nx))
    R = diagm(ones(nu))

    # variables we are solving for
    X = cvx.Variable(nx,N_mpc)
    U = cvx.Variable(nu,N_mpc-1)

    # TODO: implement cost function
    obj = 0
    for k = 1:N_mpc-1
        obj += 0.5*cvx.quadform(X[:,k] - X_ref_window[k], Q) + 0.5*cvx.quadform(U[:,k], R)
    end

    obj += 0.5*cvx.quadform(X[:,N_mpc] - X_ref_window[N_mpc], Q)
    

    # create problem with objective
    prob = cvx.minimize(obj)

    # TODO: add constraints with prob.constraints += 
    prob.constraints += (X[:,1] == xic)
    for i = 1:N_mpc-1
        prob.constraints += (X[:,i+1] == A*X[:,i] + B*U[:,i])
        prob.constraints += (U[:,i]<=u_max)
        prob.constraints += (U[:,i]>=u_min) 
        prob.constraints += (X[2,i] <= xg[2])
    end
    prob.constraints += (X[2,N_mpc] <= xg[2])
    prob.constraints += (X[:,N_mpc] == xg)

    

    # solve problem 
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = true)

    # get X and U solutions 
    X = X.value
    U = U.value
    
    # return first control U 
    return U[:,1]
end


function extended_kalman_filter(xi, 
                                xg,
                                u, 
                                Ad,
                                Bd)

    # get our sizes for state and control
    nx = length(xi)
    nu = length(u)


    Q_kf = 0.01 * Matrix{Float64}(I, nx, nx)  # Process noise covariance
    R_kf = 0.1 * Matrix{Float64}(I, nx, nx)   # Measurement noise covariance

    function f(x, u)
        return Ad*x + Bd*u
    end

    function h(x)
        return x
    end

    x_pred = f(xi, u)
    F = FD.jacobian(f, xi, u)
    P_pred = F * Q_kf * F' + R_kf

    y = xg - h(x_pred)
    H = FD.jacobian(h, x_pred)
    S = H * P_pred * H' + R_kf
    K = P_pred * H' * inv(S)

    xi = x_pred + K * y
    P = (Matrix{Float64}(I, nx, nx) - K * H) * P_pred
    
    @assert size(xi) == (nx,)

    return xi

end



@testset "convex mpc" begin 

    # create our discrete time model 
    dt = 1.0
    A,B = create_dynamics(dt)
    # println(size(A))
    # println(size(B))

    # Ad, Bd = create_dynamics(dt)

    # get our sizes for state and control
    nx,nu = size(B)

    # initial and goal states
    x0 = [-2;-4;2;0;0;.0]
    xg = [0,-.68,3.05,0,0,0]

    # bounds on U
    u_max = 0.4*ones(3)
    u_min = -u_max

    # problem size and reference trajectory 
    N = 100 
    t_vec = 0:dt:((N-1)*dt)
    X_ref = [desired_trajectory(x0,xg,N,dt)...,[xg for i = 1:N]...] 
    
    # MPC window size 
    N_mpc = 20 
    
    # sim size and setup 
    N_sim = N + 20 
    t_vec = 0:dt:((N_sim-1)*dt)
    X_sim = [zeros(nx) for i = 1:N_sim]
    X_sim[1] = x0 
    U_sim = [zeros(nu) for i = 1:N_sim-1]
    
    # simulate 
    @showprogress "simulating" for i = 1:N_sim-1 
        
        # get state estimate

        #1. State Estimation: from homework
        #xi_estimate = state_estimate(X_sim[i], xg)

        # xi_test = state_estimate(X_sim[i], xg)
        # size(xi_test)

        xi_estimate = X_sim[i]

        # #2. EKF for state estimation
        xi_estimate = extended_kalman_filter(xi_estimate, xg, U_sim[i], A, B)
  
        
        # TODO: given a window of N_mpc timesteps, get current reference trajectory
        X_ref_tilde = X_ref[i:i+N_mpc-1]
        
        # TODO: call convex mpc controller with state estimate 
        u_mpc = convex_mpc(A,B,X_ref_tilde,xi_estimate,xg,u_min,u_max,N_mpc)
        
        # commanded control goes into thruster model where it gets modified 
        U_sim[i] = thruster_model(X_sim[i], xg, u_mpc)
        
        # simulate one step 
        X_sim[i+1] = A*X_sim[i] + B*U_sim[i]
    end


end

convex mpc: Error During Test at /home/peter/Desktop/16-745/Assignments/HW2_S24-main/Project_Space.ipynb:115
  Got exception outside of a @test
  DimensionMismatch("A has dimensions (6,3) but B has dimensions (6,6)")
  Stacktrace:
    [1] gemm_wrapper!(C::Matrix{Float64}, tA::Char, tB::Char, A::Matrix{Float64}, B::Matrix{Float64}, _add::LinearAlgebra.MulAddMul{true, true, Bool, Bool})
      @ LinearAlgebra /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/matmul.jl:643
    [2] mul!
      @ /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/matmul.jl:169 [inlined]
    [3] mul!
      @ /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/matmul.jl:275 [inlined]
    [4] *
      @ /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/matmul.jl:160 [inlined]
    [5] *
      @ ./operators.jl:560 [inlined]
    [6] extended_kalman_filter(xi::Vector{Flo

TestSetException: Some tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.